# CrewAI Agents for Blogger Support

Shows how CrewAI can coordinate multiple agents to help generate and refine blog content.

In [ ]:
!pip install crewai crewai_tools python_dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
os.environ["OPENAI_MODEL_NAME"]='gpt-4o-mini'

In [ ]:
from crewai import Agent, Task, Crew

# AI Agent to support creation of articles

##Creating Agents
Define Agents and provide a role, goal and backstory

In [ ]:
planner = Agent(
    role="Content planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
                "about the topic {topic}."
                "You collect information that helps the "
                "audience learn something and make informed decisions "
                "Your work is the basis for the Content Writer to write an article on the topic"
                "Content should be written in italian language",
    allow_delegation=False,
    verbose=True
)

In [ ]:
writer = Agent(
    role="Content writer",
    goal="Write insightful and factually accurate opinion piece about the topic {topic}",
    backstory="You're working on a writing a new opinion piece about the topic {topic} "
                "You base your writing on the work of the COntent Planner, who provides an outline "
                "and relevant context about the topic. "
                "You follow the main objectives and direction of the outline, as provided to you by the Content Planner. "
                "You also provide objective and impartial insights and back them up with information provide by Content Planner."
                "You acknowledge in your opinion piece when your statements are opiniions as opposed to objective statements."
                "Article should be written in italian language",
    allow_delegation=False,
    verbose=True
)

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization",
    backstory="You are an editor who receives a blog post from the COntent Writer. "
                "Your goal is to review the blog post to ensure that is follows journalistic best practices, "
                "provide balanced viewpoints when providing opinions or asssertions, "
                "and also avoid major controversial topics or opinions when possibile "
                "Article should be written in italian langue",
    allow_delegation=False,
    verbose=True
)

##Defining Tasks
Defining tasks and provide them a description, expected_output and agent

In [ ]:
plan = Task(
    description=(
        "1. Prioritize latest trends on topic {topic}\n"
        "2. Identify the target audience considering their interests.\n"
        "3. Develop a detailed content outline including intro, key points and a call to action.\n"
        "4. Include SEO keywords and relevant sources"
    ),
    expected_output="A comprehensive content plan document woth outline, audience analysis, SEO keywords and resources",
    agent=planner
)

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}\n"
        "2. Incorporate SEO keywords naturally\n"
        "3. Ensure the post is structured with an engagin intro, insightful body and summarizing conclusion."
    ),
    expected_output="A well written blog post in markdow format ready for publication. Each section should have 2 or 3 paragraphs.",
    agent=writer
)

In [ ]:
edit = Task(
    description=(
        "Review the given blog post for grammatical errors and alignment with the brand's voice that is a journal for teenagers."
    ),
    expected_output="A well written blog post in markdow format ready for publication. Each section should have 2 or 3 paragraphs.",
    agent=editor
)

##Creating the Crew
Create crew of agents and pass them the tasks.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Stipendio dei giocatori di calcio"})

In [ ]:
output = result.raw
output = output.replace('```markdown', '')
output = output.replace('```', '')

In [ ]:
from IPython.display import display, Markdown
Markdown(output)

# AI Agent to support aggregation of info from websites

Scrapping tool

In [ ]:
from crewai_tools.tools import ScrapeWebsiteTool

scrape_tool = ScrapeWebsiteTool()
scrape_tool = ScrapeWebsiteTool(website_url='https://www.repubblica.it/')

text = scrape_tool.run()
print(text[:500])

Sampple of Custom tool
*   Agent level: Agent can use the tool on any task performed
*   Task level: Agent can use the tool when performing that specific task (overrides previous one)

In [ ]:
from langchain.tools import tool

@tool("Calculator")
def calculate(operation):
  """
    Useful to perform any math calculations, like sum, minus, multiplication and division.
    The input to this tool should be a mathematical expression, a coouple of example are `200*7` or `50/2`
  """

  try:
    return eval(operation)
  except Exception as e:
    return "Error: Invalid syntax in mathematical expression"

##Creating Agents with relative Tasks

In [ ]:
newspaper_reader_agent = Agent(
    role="NewsPaperReader",
    goal="Collect news from all websites and summarize them",
    backstory=(
        "You work for an important communication company and you are asked to read all news from relevant websites. "
        "You should provide clear and concise summary of all the news. Use language Italian."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
newspapaer_reading = Task(
    description=(
        "Read all the news from the following websites: {websites}\n"
        "Summarize the news in a bullet point list\n"
    ),
    expected_output="A bullet point list of the main news from websites.",
    tools=[scrape_tool],
    agent=newspaper_reader_agent
)

In [ ]:
newspaper_filter_agent = Agent(
    role="NewsPaperFilter",
    goal="Extract only news related to specific topics",
    backstory=(
        "You work for an important communication company and you are asked to find all news from all relevant websites "
        "that are related to a specific topics. Use language Italian."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
newspapaer_filter = Task(
    description=(
        "Read news summary and extract only news related to the following topics: {topics}"
        "Summarize the news in a bullet point list\n"
    ),
    expected_output="A bullet point list of the main news for each provided topic.",
    agent=newspaper_filter_agent
)

In [ ]:
crew = Crew(
    agents=[newspaper_reader_agent, newspaper_filter_agent],
    tasks=[newspapaer_reading, newspapaer_filter],
    verbose=True ,
    memory=True
)

In [ ]:
inputs = {
    "websites": ["https:www.repubblica.it/", "https:www.ilsole24ore.com/"],
    "topics": ["Economia", "Attualità"]
}

result = crew.kickoff(inputs=inputs)

In [ ]:
output = result.raw

In [ ]:
from IPython.display import display, Markdown
Markdown(output)